### Fig. 2: Label Projection from 3D Point Clouds

In [ ]:
from pathlib import Path

import numpy as np
from PIL import Image
import rasterio

In [ ]:
color_palette = [
    [209, 64, 129, 255],  # other purple
    [128, 255, 144, 255],  # light green
    [219, 254, 135, 255],  # mindaro
    [62, 137, 20, 255],  # inida green
    [255, 252, 49, 255],  # yellow
    [140, 192, 132, 255],  # dark sea green # [245, 93, 62, 255],  # orange
    [105, 116, 124, 255],  # slate gray
    [19, 70, 17, 255],  # forest green
    [125, 97, 103, 255],  # deep taube
    [255, 180, 0, 255],  # yellow
    [6, 167, 125, 255],  # green
    [129, 164, 205, 255],  # iceberg
    [75, 29, 63, 255],  # purple
    [247, 197, 72, 255],  # mayze crayola
    [62, 124, 177, 255],  # steel blue
    [102, 99, 112, 255],  # burnished brown
    [117, 92, 27, 255],  # field drab
    [115, 191, 184, 255],  # turquoise
    [73, 67, 49, 255],  # olive drab
    [163, 0, 0, 255],  # rufous
    [163, 147, 191, 255],  # glossy grape
    [111, 115, 210, 255],  # violet blue crayola
    [179, 106, 94, 255],  # copper penny
    [0, 178, 202, 255],  # pacific blue
    [118, 117, 34, 255],  # spanish bistre
    [238, 215, 197, 255],  # champagne pink
    [86, 53, 30, 255],  # van dyke brown
]

#### Visualize label image

In [ ]:
base_dir = Path("G:/3D-GeoInfo-2025/data/")
image_path = base_dir / "1_base_data/labels/automatic_labeling/s1_p1_small_labels.tif"

with rasterio.open(image_path) as image:
    image_array = image.read()

rgb_image_array = np.full((image_array.shape[1], image_array.shape[2], 3), fill_value=240, dtype=np.uint8)

for idx, instance_id in enumerate(np.unique(image_array)):
    if instance_id == 0:
        continue
    instance_mask = (image_array == instance_id).reshape((image_array.shape[1], image_array.shape[2]))
    color = color_palette[idx % len(color_palette)]
    rgb_image_array[instance_mask] = color[:3]

image = Image.fromarray(rgb_image_array, mode="RGB")
image.save("./label_projection.png")

#### Colorize point cloud

In [ ]:
from pointtorch import read
from pointtorch.operations.numpy import make_labels_consecutive

point_cloud_path = base_dir / "1_base_data/point_clouds/s1_p1_small.laz"
point_cloud = read(point_cloud_path)
instance_ids = make_labels_consecutive(point_cloud["instance_id_prediction"].to_numpy(), ignore_id=0)

point_cloud["red"] = 240
point_cloud["green"] = 240
point_cloud["blue"] = 240

for idx, instance_id in enumerate(np.unique(instance_ids)):
    if instance_id == 0:
        continue
    instance_mask = instance_ids == instance_id
    color = color_palette[idx % len(color_palette)]
    point_cloud.loc[instance_mask, ["red", "green", "blue"]] = np.array(color)[:3]

point_cloud.to("./point_cloud_colored.laz")